In [ ]:
import spacy
import json
import zipfile
import os
import tensorflow as tf
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, Dropout, LeakyReLU, Activation, ZeroPadding2D
from keras.models import Model, Sequential
import vocab

In [4]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore

# example caption
caption = "A red sports car parked on a city street."
nlp = spacy.load("en_core_web_sm")
# define tokenizer and fit on captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts([caption])

# convert caption into numerical array
sequences = tokenizer.texts_to_sequences([caption])

# define embedding matrix
word_index = tokenizer.word_index
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

# define numerical array
max_length = 10
padded_seq = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

# print numerical array
print(padded_seq)

[[1 2 3 4 5 6 1 7 8 0]]


In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anisn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anisn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np

def extract_features(img_path):
    # Load the pre-trained CNN model
    model = VGG16(weights='imagenet', include_top=False)

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224)) # type: ignore
    img = image.img_to_array(img) # type: ignore
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # Extract features from the image
    features = model.predict(img)
    features = np.reshape(features, (features.shape[0], -1))

    return features


In [10]:
import tensorflow as tf
import keras
from keras import layers
from SAE_model import *



# # Define the autoencoder model
# autoencoder = Model(encoder_inputs, decoder_outputs)

# # Compile the model
# autoencoder.compile(optimizer='adam', loss='mse')


In [11]:
import torch
import numpy as np

def encode_decode(caption_str, autoencoder):
    # Convert the caption to a tensor
    caption_tensor = torch.tensor(np.array([extract_features(caption_str)]))

    # Pass the caption through the autoencoder
    encoded = autoencoder.encoder(caption_tensor)
    decoded = autoencoder.decoder(encoded)
    vocab_idx2word = {i: w for w, i in decoded[0]}

    # Convert the decoded tensor back to a string
    caption = ' '.join([vocab_idx2word[i] for i in decoded[0]])



    return caption


In [12]:
import nltk
from nltk.tokenize import word_tokenize

def extract_important_words(caption):
    # Tokenize the caption into words
    words = word_tokenize(caption)

    # Define the set of stop words to remove
    stop_words = set(nltk.corpus.stopwords.words("english"))

    # Remove stop words and non-alphabetic characters
    important_words = [word.lower() for word in words if word.lower() not in stop_words and word.isalpha()]

    return important_words

In [ ]:
# Open the zip file containing the input files
with zipfile.ZipFile("data/train2017.zip", "r") as zip_ref:
    
    for file_name in zip_ref.namelist():
        
        if file_name.endswith(".txt"):
            # Read the contents of the file
            with zip_ref.open(file_name) as file:
                caption = file.read().decode('utf-8')
                caption = encode_decode(caption,autoencoder)
            # call the imp function
            important_words = extract_important_words(caption)
            # Create a dictionary 
            data = {
                "file_name": file_name,
                "important_words": important_words
            }
            # Convert the dictionary to JSON format
            json_data = json.dumps(data)

            
            json_file_name = os.path.splitext(file_name)[0] + ".json"
            # save the JSON file
            with zip_ref.open(json_file_name, "w") as json_file:
                json_file.write(json_data.encode('utf-8'))

# Add the JSON files to the zip folder
with zipfile.ZipFile("data/train2017.zip", "a") as zip_ref:
    # Loop through all the JSON files in the directory
    for file_name in os.listdir():
        if file_name.endswith(".json"):
            # Add the JSON file to the zip folder
            zip_ref.write(file_name)
            # Remove the JSON file from the directory to avoid redundancy
            os.remove(file_name)

In [ ]:
import os

folder_path = "C:/Users/Anisn/Documents/Final Project/data/train2017/images"

images = os.listdir(folder_path)

print(images[:100])

path = "C:/Users/Anisn/Documents/Final Project/data/train2017/captions"
file_names = os.listdir(path)

captions = []
for file_name in file_names[:100]:
    if file_name.endswith(".txt"):
        with open(os.path.join(path, file_name), "r") as file:
            text = file.read()
            captions.append(text)

In [ ]:
important_words = []
for i in captions:
    important_words.append(extract_important_words(i))

In [ ]:
import spacy
import nltk
from nltk.tokenize import word_tokenize
nlp = spacy.load("en_core_web_sm")

def extract_important_words(caption):
    # Tokenize the caption into words
    words = word_tokenize(caption)

    # Define the set of stop words to remove
    stop_words = set(nltk.corpus.stopwords.words("english"))

    # Remove stop words and non-alphabetic characters
    important_words = [word.lower() for word in words if word.lower() not in stop_words and word.isalpha()]

    return important_words

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def process_caption(caption):
    doc = nlp(caption)
    important_words = []
    for token in doc:
        if token.pos_ in ["NOUN", "VERB", "ADJ"]:  # extract only NOUN, VERB, ADJ
            important_words.append(token.text.lower())
    # convert words to numerical values
    word_to_num = {'car': 1, 'truck': 2, 'motorbike': 3, 'bus': 4, 'vehicle': 5, 'street': 6, 'road': 7, 'city': 8, 'forest': 9, 'tree': 10, 'suv': 11, 'house': 12, 'lawn': 13, 'flower': 14, 'skyline': 15, 'school': 16, 'neighborhood': 17, 'tractor': 18, 'hay': 19, 'baler': 20, 'field': 21, 'wheat': 22, 'sun': 23, 'hill': 24, 'convertible': 25, 'coastal': 26, 'ocean': 27, 'view': 28, 'palm': 29, 'delivery': 30, 'warehouse': 31, 'worker': 32, 'cargo': 33, 'box': 34, 'pallet': 35, 'rv': 36, 'campground': 37, 'forest': 38, 'lake': 39, 'bird': 40, 'chirping': 41, 'motorcycle': 42, 'people': 43, 'traffic': 44, 'bustling': 45}
    num_array = [word_to_num[word] for word in important_words if word in word_to_num]
    return num_array
processed_captions = [process_caption(caption) for caption in captions]

print(processed_captions)

In [ ]:
import json
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import_words = []

for i in captions:
  import_words.append(extract_important_words(i))

# print(import_words)
data = []
i = [i for i in range(100)]
for a,b,c,d,e in zip(images,captions, i, import_words, processed_captions):
  data.append({"IMG" : a,"caption": b,"idx": c ,"IMP": d, "values": e})

# Save data to JSON file
with open("saved_data/captions_info.json", "w") as outfile:
    json.dump(data, outfile)